Problem Statement The retail and warehouse data provided contains sales and transfer information across different suppliers, items, and time periods (months and years). The goal of the problem might be:

Sales Forecasting: Predict future sales (retail or warehouse) based on historical data. This is a Time Series Forecasting problem.

Demand Prediction: Estimate the demand for items based on the available sales and transfer data to optimize inventory management.

Supplier-Item Analysis: Identify patterns and trends in supplier performance, specific item sales, and warehouse transfers to improve logistics or retail strategies.

Sales Transfer Prediction: Predict how items are transferred between retail and warehouse locations to balance inventory levels. Depending on the specific goal, different models and techniques like time series forecasting, regression, or even classification (for items at risk of running out of stock) might be used.

Variable:

YEAR: Type: Categorical (or could be continuous depending on use) Description: The year in which the data was recorded. ML Perspective: It can be used for temporal analysis or as a time feature in forecasting.

MONTH: Type: Categorical Description: The month in which the data was recorded (e.g., January, February). ML Perspective: Another temporal feature, typically used in combination with the year for seasonality detection in sales.

SUPPLIER: Type: Categorical Description: The supplier providing the product. ML Perspective: Can be used to determine supplier performance or to cluster items based on supplier contribution.

ITEM CODE: Type: Categorical Description: Unique identifier for each item/product. ML Perspective: Acts as a key or identifier for individual items, useful in item-specific modeling or clustering.

ITEM DESCRIPTION: Type: Categorical Description: The descriptive name of the item/product. ML Perspective: This feature might be irrelevant for modeling as it could be redundant with the ITEM CODE, unless NLP techniques are applied for item classification or extraction of useful information.

ITEM TYPE: Type: Categorical Description: Type or category of the item (e.g., electronics, clothing, food). ML Perspective: Important for categorical encoding and for determining different sales patterns across item types.

RETAIL SALES (RETAIL SA): Type: Continuous Description: The total retail sales for the specific item. ML Perspective: This is a key feature for predictive models, especially in sales forecasting.

RETAIL TRANSFERS (RETAIL TR): Type: Continuous Description: The quantity of items transferred between retail locations. ML Perspective: Useful for predicting transfer needs or optimizing inventory balancing between stores.

WAREHOUSE SALES: Type: Continuous Description: The total sales from warehouse inventory. ML Perspective: Useful in understanding how much of the stock is sold directly from the warehouse, which could be compared with retail sales for demand planning.

Potential ML Tasks Predicting retail or warehouse sales: Based on item, supplier, and temporal data. Classifying items with high transfer rates: To manage inventory effectively. Time Series Analysis: To identify trends, seasonality, and forecast future sales.

These features allow us to solve multiple ML problems such as forecasting, classification, or clustering depending on business needs.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('retail_warehouse_data.csv')
df.shape

(30000, 9)

In [5]:
df.head()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0


In [3]:
df.columns

Index(['YEAR', 'MONTH', 'SUPPLIER', 'ITEM CODE', 'ITEM DESCRIPTION',
       'ITEM TYPE', 'RETAIL SALES', 'RETAIL TRANSFERS', 'WAREHOUSE SALES'],
      dtype='object')

In [6]:
df.duplicated().sum()

0

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   YEAR              30000 non-null  int64  
 1   MONTH             30000 non-null  int64  
 2   SUPPLIER          29967 non-null  object 
 3   ITEM CODE         30000 non-null  object 
 4   ITEM DESCRIPTION  30000 non-null  object 
 5   ITEM TYPE         30000 non-null  object 
 6   RETAIL SALES      29999 non-null  float64
 7   RETAIL TRANSFERS  30000 non-null  float64
 8   WAREHOUSE SALES   30000 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 2.1+ MB


In [9]:
df.describe()

,YEAR,MONTH,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
count,30000.0,30000.000000,29999.000000,30000.000000,30000.000000
mean,2020.0,3.911467,6.939796,6.594058,27.431031
std,0.0,2.836788,33.081054,27.879428,272.166085
min,2020.0,1.000000,-0.420000,-6.000000,-3999.000000
25%,2020.0,1.000000,0.000000,0.000000,0.000000
50%,2020.0,3.000000,0.160000,0.000000,1.000000
75%,2020.0,7.000000,2.920000,3.000000,6.000000
max,2020.0,9.000000,2739.000000,1507.000000,18317.000000


In [10]:
df.nunique()

YEAR                    1
MONTH                   4
SUPPLIER              290
ITEM CODE           15668
ITEM DESCRIPTION    15732
ITEM TYPE               8
RETAIL SALES         3657
RETAIL TRANSFERS      542
WAREHOUSE SALES      1032
dtype: int64

In [13]:
df.isnull().sum()

YEAR                 0
MONTH                0
SUPPLIER            33
ITEM CODE            0
ITEM DESCRIPTION     0
ITEM TYPE            0
RETAIL SALES         0
RETAIL TRANSFERS     0
WAREHOUSE SALES      0
dtype: int64

In [15]:
(df.isnull().sum()/len(df))*100

YEAR                0.00
MONTH               0.00
SUPPLIER            0.11
ITEM CODE           0.00
ITEM DESCRIPTION    0.00
ITEM TYPE           0.00
RETAIL SALES        0.00
RETAIL TRANSFERS    0.00
WAREHOUSE SALES     0.00
dtype: float64

In [11]:
df['RETAIL SALES'].fillna(df['RETAIL SALES'].median(),inplace=True)

In [12]:
df.isnull().sum()

YEAR                 0
MONTH                0
SUPPLIER            33
ITEM CODE            0
ITEM DESCRIPTION     0
ITEM TYPE            0
RETAIL SALES         0
RETAIL TRANSFERS     0
WAREHOUSE SALES      0
dtype: int64

In [16]:
df['SUPPLIER'].fillna('UNKNOWN',inplace=True)